In [13]:
import torch
import os
import random
import numpy as np
import albumentations as A
import pandas as pd
from albumentations.pytorch import ToTensorV2
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from Cyto_dataset import Cyto_train_dataset, Cyto_test_dataset
from torchvision import models
import torch.nn as nn
from utils import *
import torchvision
import torchvision.transforms.functional as TF
import scipy as sp
import time
from albumentations.core.transforms_interface import DualTransform, ImageOnlyTransform

import torchvision
import time

from classifier_utils_sw import submit_probs, out_label
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

seed=20
seed_everything(seed)


batch_size = 64
image_size = 320


metric = ['acc', 'spe', 'sensi', 'pre', 'npv', 'f1']

def eval(y_true, y_pred):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    try: acc = (tp + tn) / (tp + tn + fp + fn)    
    except: acc = 0
    try: spe = tn / (tn + fp) # specificity
    except: spe = 0
    try: sensi = tp / (tp + fn) # sensitivity, recall    
    except: sensi = 0
    try: pre = tp / (fp + tp) # precision
    except: pre = 0
    try: npv = tn / (tn + fn)    
    except: npv=0
    try: f1 = 2 * (pre * sensi) / (pre + sensi)    
    except: f1 = 0
 
    return [acc, spe, sensi, pre, npv, f1]


class Test_Rotation(DualTransform):
    """for tta"""
    def __init__(self, angle, always_apply=False, p=0.5):
        self.angle = angle
        self.always_apply = always_apply
        self.p = p

        super(Test_Rotation, self).__init__(always_apply, p)
        self.mask_value = None

    def apply(self, image, **params):
        # print(image.shape)
        return sp.ndimage.rotate(image, self.angle, axes=(0,1), order=1,
                reshape=False, mode='reflect')

    def get_transform_init_args_names(self):
        return ()

class Test_Shift(DualTransform):
    """for tta"""
    def __init__(self, shift, always_apply=False, p=0.5):
        self.shift = shift + [0]
         # 0(h) or 1(w)
        self.always_apply = always_apply
        self.p = p

        super(Test_Shift, self).__init__(always_apply, p)
        self.mask_value = None

    def apply(self, image, **params):
        # print(image.shape)
        return sp.ndimage.shift(image, self.shift, order=1, mode='reflect')

    def get_transform_init_args_names(self):
        return ()


scale = A.Resize(image_size, image_size)
scale2 = A.Resize(344, 344)
#rcrop = A.RandomCrop(width=256, height=256)
shift_w = Test_Shift([0,10], p=1)
shift_h = Test_Shift([10,0], p=1)
rotate1 = Test_Rotation(5, p=1)
rotate2 = Test_Rotation(-15, p=1)
H_flip = A.HorizontalFlip(p=1)
V_flip = A.VerticalFlip(p=1)

normalize = A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
to_tensor = ToTensorV2()

test_transform = A.Compose(
        [
            scale,
            #scale_te,
            normalize,
            to_tensor
        ])


train_list, val_list = train_val_split('/home/ec2-user/dataset/b-trac-cyto/final', train_ratio=0.9)
#train_list, val_list = train_val_split('/home/ec2-user/dataset/b-trac-cyto/practice', train_ratio=0.0)

val_set = Cyto_train_dataset(val_list, transform=test_transform)

val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=4)


file_name = 'image_size_320_cos_full/14'

model = models.resnet50(pretrained=False)
model.avgpool = nn.AdaptiveAvgPool2d((1, 1))
model.fc = nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load( '/home/ec2-user/workspace/check_point/' + file_name + '.pth'))
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")
model.to(device)

out_label = out_label(model, val_loader, device)
#print(out_label(model, val_loader, device))


make_csv = True
save_dir = './result/'
createFolder(save_dir)
print('batch_size', batch_size)
# transform_list = [None, V_flip, H_flip, rotate1, rotate2, shift_w, shift_h] # 0, 1, 2, 3, 4
transform_list = [None, V_flip, H_flip]
#transform_list = [rotate1, rotate2] 
# t_name_list = ['o', 'V', 'H','r15','r' ,'sw10', 'sh' ]
t_name_list = ['o', 'V', 'H']
t_name_len = len(t_name_list)
cur_t_name = ''

probs = 0 # 누적 probs
print('TTA submit')
for idx, new_transform in enumerate(transform_list):
    if new_transform is None:
        new_transform = [scale]
    elif new_transform == 's2':
        new_transform = [scale2]
    else:
        new_transform = [scale, new_transform]
    cur_t_name += t_name_list[idx]
    test_transform = A.Compose( new_transform +  [normalize, to_tensor] )
    print('transform', new_transform)
    test_set = Cyto_test_dataset(val_list, test_transform)
    test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=4)
    print(f'num_test : {len(test_loader)}')

    dic = submit_probs(model, test_loader, device)
    probs += dic['probs']
    print(len(probs))
    pred = np.where(probs/(idx+1) > 0.5, 1, 0)
    #pred = np.where(probs > 0.5, 1, 0)
    metric_list = eval(out_label, pred)
    avg_metric = sum(metric_list) / len(metric_list)
    
    print('f1:', f1_score(out_label, pred))
    print('acc:', accuracy_score(out_label, pred))
    print('avg_metric:', avg_metric)
    print('metric_list',metric_list)


print('TTA submit2')
scale = A.Resize(344, 344)

file_name2 = 'image_size_320_cos/14'

model = models.resnet50(pretrained=False)
model.avgpool = nn.AdaptiveAvgPool2d((1, 1))
model.fc = nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load( '/home/ec2-user/workspace/check_point/' + file_name2 + '.pth'))
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")
model.to(device)

transform_list = [None, H_flip]
t_name_list = ['o', 'H']

for idx, new_transform in enumerate(transform_list):
    if new_transform is None:
        new_transform = [scale]
    elif new_transform == 's2':
        new_transform = [scale2]
    else:
        new_transform = [scale, new_transform]
        
    cur_t_name += t_name_list[idx]
    test_transform = A.Compose( new_transform +  [normalize, to_tensor] )
    print('transform', new_transform)
    test_set = Cyto_test_dataset(val_list, test_transform)
    test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=4)
    print(f'num_test : {len(test_loader)}')

    dic = submit_probs(model, test_loader, device)
    probs += dic['probs']
    print(len(probs))
    pred = np.where(probs/(idx+1+t_name_len) > 0.5, 1, 0)
    #pred = np.where(probs > 0.5, 1, 0)
    metric_list = eval(out_label, pred)
    avg_metric = sum(metric_list) / len(metric_list)
    
    print('f1:', f1_score(out_label, pred))
    print('acc:', accuracy_score(out_label, pred))
    print('avg_metric:', avg_metric)
    print('metric_list',metric_list)
    
print("done")

seed : 20
neg_listdir :  4000 pos_listdir :  4000
len train :  7200 len val :  800
batch_size 64
TTA submit
transform [Resize(always_apply=False, p=1, height=320, width=320, interpolation=1)]
num_test : 13
(800, 2)
800
f1: 0.9860583016476552
acc: 0.98625
avg_metric: 0.9863407185633358
metric_list [0.98625, 1.0, 0.9725, 1.0, 0.9732360097323601, 0.9860583016476552]
transform [Resize(always_apply=False, p=1, height=320, width=320, interpolation=1), VerticalFlip(always_apply=False, p=1)]
num_test : 13
(800, 2)
800
f1: 0.984848484848485
acc: 0.985
avg_metric: 0.9850394400184318
metric_list [0.985, 0.995, 0.975, 0.9948979591836735, 0.9754901960784313, 0.984848484848485]
transform [Resize(always_apply=False, p=1, height=320, width=320, interpolation=1), HorizontalFlip(always_apply=False, p=1)]
num_test : 13
(800, 2)
800
f1: 0.9860935524652339
acc: 0.98625
avg_metric: 0.9863060216595984
metric_list [0.98625, 0.9975, 0.975, 0.9974424552429667, 0.9755501222493888, 0.9860935524652339]
TTA submit2